In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

### Sales Funnel Generator

This tool can generate a sales funnel, that shows the current state of Leads and Opportunities (exported from SalesForce), based on a date range that selects when the leads / opportunities were created.

Note that the chart often displays slightly strangely; this is due to inconsistencies in the data itself.

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import pandas as pd
import numpy as np
from IPython.display import (Image,SVG)
import ipywidgets as widgets
sns.set()
plt.rcParams['figure.figsize'] = [12,9]
plt.rc("xtick", labelsize=13)
plt.rc("ytick", labelsize=13)
plt.rc("axes", titlesize=24, labelsize=16)
plt.rc("legend", fontsize=14)

from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
layout = Layout(width="1000", height="700")
def sankey(**value):
    """Show SankeyWidget with default values for size and margins"""
    return SankeyWidget(layout=layout, margins=dict(top=10, bottom=10, left=130, right=130), **value)

### Read & Prep Data

# Opportunities report
opportunities = pd.read_csv("salesforce_opportunities.csv", delimiter=";")
opportunities["Amount"] = opportunities["Amount"].apply(lambda x: int(str(x)[:-3]) if str(x)!='nan' else 0)
opportunities["Stage Duration"] = opportunities["Stage Duration"].apply(lambda x: int(x[:-11]))
opportunities["Age"] = opportunities["Age"].apply(lambda x: int(x[:-11]))
opportunities["Days to Close"] = pd.to_datetime(opportunities["Close Date"]) - pd.to_datetime(opportunities["Created Date"])
opportunities["Days to Close"] = opportunities["Days to Close"].apply(lambda x: x.total_seconds() / (60*60*24))
opportunities = opportunities.set_index("Opportunity ID")
opportunities["Created Date"] = pd.to_datetime(opportunities["Created Date"])

# Leads report
leads = pd.read_csv("salesforce_leads.csv", delimiter=';')
leads["Create Date"] = pd.to_datetime(leads["Create Date"])

In [3]:
def draw_sankey(leads, contacted, uncontacted, opportunities, sent, disqualified, won, progress, lost):

    leads_string = "Leads (%d)"%leads
    contacted_string = "Contacted (%d)"%contacted
    uncontacted_string = "Uncontacted (%d)"%uncontacted
    opportunities_string = "Opportunities (%d)"%opportunities
    sent_string = "In Contact (%d)"%sent
    disqualified_string = "Disqualified (%d)"%disqualified
    won_string = "Closed Won (%d)"%won
    progress_string = "In Progress (%d)"%progress
    lost_string = "Closed Lost (%d)"%lost

    links = [
        # Leads
        {'source':' ',              'target':leads_string,         'value':leads,         'color':'green' },
        {'source':leads_string,     'target':contacted_string,     'value':contacted,     'color':'green' },
        {'source':leads_string,     'target':uncontacted_string,   'value':uncontacted,   'color':'orange' },
        {'source':contacted_string, 'target':opportunities_string, 'value':opportunities, 'color':'green' },
        {'source':contacted_string, 'target':disqualified_string,  'value':disqualified,  'color':'red' },
        {'source':contacted_string, 'target':sent_string,          'value':sent,          'color':'orange' },  
        # Opportunities (184)
        {'source':opportunities_string, 'target':won_string,   'value':won, 'color':'green' },
        {'source':opportunities_string, 'target':progress_string, 'value':progress, 'color':'orange' },
        {'source':opportunities_string, 'target':lost_string,  'value':lost, 'color':'red' },
    ]

    order = [
        [' '],
        [leads_string],
        [contacted_string, uncontacted_string],
        [opportunities_string,sent_string,disqualified_string],
        [won_string, progress_string, lost_string]
    ]
    
    s = sankey(links=links, align_link_types=True, order=order)
    display(s)
    
    
def drawer(leads, opportunities, start_date, end_date, people):
    
    start_date = dt.datetime.combine(start_date, dt.datetime.min.time())
    end_date = dt.datetime.combine(end_date, dt.datetime.min.time())
    
    selected_leads = leads[ (leads["Create Date"] > start_date) & (leads["Create Date"] < end_date) ]
    selected_opportunities = opportunities[ (opportunities["Created Date"]>start_date) & (opportunities["Created Date"]<end_date) ]
    selected_leads = selected_leads[ selected_leads["Lead Owner"].isin(people) ]
    selected_opportunities = selected_opportunities[ selected_opportunities["Opportunity Owner"].isin(people) ]
    counts = selected_leads["Lead Status"].value_counts()
    opp_counts = selected_opportunities["Stage"].value_counts()

    leads = len(selected_leads)
    try: contacted = counts["Contacted"] + counts["Information Sent"] + counts["Qualified"] + counts["Disqualified"]
    except: contacted = 0
    try: uncontacted = counts["New"]
    except: uncontacted = 0
    try: opportunities = counts["Qualified"]
    except: opportunities = 0
    try: sent = counts["Contacted"] + counts["Information Sent"]
    except: sent = 0
    try: disqualified = counts["Disqualified"]
    except: disqualified = 0
    try: won = opp_counts["Closed Won"]
    except: won = 0
    try: progress = opp_counts["Contacted"] + opp_counts["Proposal"] + opp_counts["Contract"]
    except: progress = 0
    try: lost = opp_counts["Closed Lost"]
    except: lost = 0

    draw_sankey(leads, contacted, uncontacted, opportunities, sent, disqualified, won, progress, lost)

In [4]:
start_date = dt.datetime(2018,1,3)
end_date = dt.datetime(2018,6,30)
people = list(set(opportunities["Opportunity Owner"].unique().tolist() + leads["Lead Owner"].unique().tolist()))

people_widget = widgets.SelectMultiple(
    options=people,
    rows=len(people),
    description="Employees"
)
start_date_widget = widgets.DatePicker(
    description='Start Date',
    disabled=False,
    value=start_date
)
end_date_widget = widgets.DatePicker(
    description='End Date',
    disabled=False,
    value=end_date
)

display(people_widget)
display(start_date_widget)
display(end_date_widget)

SelectMultiple(description='Employees', options=('Ward Gilles', 'Eelco Kienhuis', 'Mitch Nijburg', 'Nick Scott…

DatePicker(value=datetime.datetime(2018, 1, 3, 0, 0), description='Start Date')

DatePicker(value=datetime.datetime(2018, 6, 30, 0, 0), description='End Date')

In [14]:
from IPython.display import Javascript, display
from ipywidgets import widgets
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Draw Graph")
button.on_click(run_all)
display(button)


# interact(drawer, leads=fixed(leads), 
#          opportunities=fixed(opportunities),
#          start_date=fixed(start_date_widget.value),
#          end_date=fixed(end_date_widget.value),
#          people=fixed(people_widget.value))

<IPython.core.display.Javascript object>

Button(description='Draw Graph', style=ButtonStyle())

In [15]:
drawer(leads, opportunities, start_date_widget.value, end_date_widget.value, people_widget.value)

SankeyWidget(align_link_types=True, layout=Layout(height='700', width='1000'), links=[{'source': ' ', 'target'…

In [16]:
# # History - only saved in 2018, so can only analyse recent data
# history = pd.read_csv("salesforce_history.csv", delimiter=';')
# stage_changes = history[ history["Field / Event"] == "Stage" ][["Old Value", "New Value","Edit Date","Opportunity ID"]]
# stage_changes = stage_changes.set_index("Opportunity ID")

# # Get IDs that are really new garages this year (manually skip some BS)
# new_IDs = opportunities[ opportunities["Created Date"] > dt.datetime(2018,1,3) ].index.unique()
# skip_IDs = [
#     '0065800000f09XZ',
#     '0065800000VNrZp',
#     '0065800000VNmx4',
#     '0065800000VOCgK',
#     '0065800000VOC8h',
#     '0065800000V2BLI',
#     '0065800000VNml3',
#     '0065800000VNoLt',
#     '0065800000VNoSu',
#     '0065800000VNoQj',
#     '0065800000VNn1z',
#     '0065800000VNmqh',
#     '0065800000VNnCn',
#     '0065800000VNmr6'
# ]
# new_IDs = [item for item in new_IDs if item not in skip_IDs]